In [ ]:
# Copyright 2023 DeepMind Technologies Limited
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Imports

In [ ]:
!pip install --upgrade https://github.com/google-deepmind/nuclease-design.git

In [ ]:
import itertools
import pandas as pd

from nuclease_design import utils
from nuclease_design import constants

# Load data

In [ ]:
DATA_DIR = constants.DATA_DIR

In [ ]:
%%time
all_data_df = utils.load_all_data(DATA_DIR)

CPU times: user 2.25 s, sys: 450 ms, total: 2.7 s
Wall time: 13.8 s


# Aggregate data across generations

In [ ]:
# By sorting and aggregating the "first", we keep the activity label from the
# most recent generation
all_data_df['generations'] = all_data_df['generation']
all_data_df = all_data_df.sort_values(by='generation', ascending=False)  # order by generation
agg_dict = {
    'sublibrary_names': lambda x: tuple(itertools.chain.from_iterable(x)),
    'generations': lambda x: tuple(sorted(x)),
    'activity_greater_than_neg_control': 'first',
    'activity_greater_than_wt': 'first',
    'activity_greater_than_a73r': 'first',
    'num_mutations': 'first',
    'sequence': 'first',
}
df = all_data_df.groupby('mutations').agg(agg_dict).reset_index()

In [ ]:
# Use the highest activity level found in the most recent generation.
def find_highest_activity_level(row, default_label='non-functional'):
  references = ['neg_control', 'wt', 'a73r']
  cols = [f'activity_greater_than_{ref}' for ref in references]
  true_indices = [
      i for i, col in enumerate(cols) if row[col] is not None and row[col]
  ]
  if not true_indices:
    return default_label
  else:
    return cols[max(true_indices)]


df['activity_level'] = (
    df.apply(find_highest_activity_level, axis=1)
    .str.replace('a73r', 'A73R')
    .str.replace('a73r', 'A73R')
    .str.replace('wt', 'WT')
    .str.replace('neg_control', '0')
)
df['is_functional'] = df['activity_level'] != 'non-functional'

In [ ]:
cols_to_keep = [
    'mutations',
    'num_mutations',
    'sublibrary_names',
    'generations',
    'activity_level',
    'is_functional',
    'sequence',
]

In [ ]:
with utils.open_file(constants.LANDSCAPE_PATH, 'w', DATA_DIR) as f:
  utils.encode_df(df[cols_to_keep]).to_csv(f, index=False)

# appendix

In [ ]:
ldf = utils.load_landscape(DATA_DIR)

Note that some variants have conflicting functional labels:

For the hypothesis tests corresponding to [neg_control, wt, a73r], there were were 5 variants with [F, T, T] (labeled as activity > A73R), and 973 variants with [F, T, F] (labeled as activity > WT),

In [ ]:
pd.crosstab(df['activity_level'], df['activity_greater_than_neg_control'])

activity_greater_than_neg_control,False,True
activity_level,,
activity_greater_than_0,0,11099
activity_greater_than_A73R,5,194
activity_greater_than_WT,973,10572
non-functional,32917,0


In [ ]:
pd.crosstab(df['activity_level'], df['activity_greater_than_wt'])

activity_greater_than_wt,False,True
activity_level,,
activity_greater_than_0,11099,0
activity_greater_than_A73R,0,199
activity_greater_than_WT,0,11545
non-functional,32917,0


In [ ]:
pd.crosstab(df['activity_level'], df['activity_greater_than_a73r'])

activity_greater_than_a73r,False,True
activity_level,,
activity_greater_than_0,8442,0
activity_greater_than_A73R,0,199
activity_greater_than_WT,10914,0
non-functional,14216,0


In [ ]:
df[['activity_greater_than_neg_control', 'activity_greater_than_wt', 'activity_greater_than_a73r']].value_counts()

activity_greater_than_neg_control  activity_greater_than_wt  activity_greater_than_a73r
False                              False                     False                         14216
True                               True                      False                          9941
                                   False                     False                          8442
False                              True                      False                           973
True                               True                      True                            194
False                              True                      True                              5
dtype: int64

In [ ]:
# 20K variants were not assessed for activity relative to A73R
# these correspond to variants seen only in g1/g2.
df[df['activity_greater_than_a73r'].isna()].generations.explode().value_counts()

g2    14219
g1     8670
Name: generations, dtype: int64

In [ ]:
# 5 variants conflict in their a73r + wt vs neg_control label.
df[df['activity_greater_than_a73r'] & df['activity_greater_than_wt'] & ~df['activity_greater_than_neg_control']].generations.value_counts()

(g3,)    4
(g4,)    1
Name: generations, dtype: int64

In [ ]:
# 973 variants conflict in their wt vs neg_control label.
df[df['activity_greater_than_wt'] & ~df['activity_greater_than_neg_control']].generations.value_counts()

(g4,)           660
(g3,)           303
(g2, g4)          3
(g1, g2, g4)      3
(g1, g3)          3
(g3, g4)          2
(g2, g3)          2
(g1, g3, g4)      1
(g2, g3, g4)      1
Name: generations, dtype: int64